In [1]:
puts `ls -l`

total 2956
-rw-rw-r-- 1 osboxes osboxes     504 Jun 30 10:49 Drug-Disease Graphing.ipynb
-rw-rw-r-- 1 osboxes osboxes  138187 Jun 27 15:44 drug-mapping.ipynb
-rw-rw-r-- 1 osboxes osboxes 2778831 Jun 19 13:41 gene disease.ipynb
drwxrwxr-x 2 osboxes osboxes    4096 Jun 30 10:50 mappings
drwxrwxr-x 2 osboxes osboxes    4096 Jun 27 15:13 raw-data
-rw-rw-r-- 1 osboxes osboxes   90846 Jun 20 10:59 therapeutic_area.ipynb


In [13]:
puts `head -5 ./mappings/drug-mappings.csv`

demokratisid,xref,demokratis_label,pubchem_cid,IUPACname
C0613621,http://purl.bioontology.org/ontology/MESH/C030536,"2,2-dichloro-1,1-difluoroethyl difluoromethyl ether",https://pubchem.ncbi.nlm.nih.gov/compound/152803,"2,2-dichloro-1,1-difluoroethyl%20difluoromethyl%20ether"
C0042291,http://purl.bioontology.org/ontology/SNOMEDCT/387080000,Valproic Acid,https://pubchem.ncbi.nlm.nih.gov/compound/3121,Valproic%20acid
C0059747,http://purl.bioontology.org/ontology/SNOMEDCT/83298009,ethyl acetate,https://pubchem.ncbi.nlm.nih.gov/compound/8857,Ethyl%20acetate
C0059747,http://purl.bioontology.org/ontology/MESH/C007650,ethyl acetate,https://pubchem.ncbi.nlm.nih.gov/compound/8857,ethyl%20acetate


In [21]:
puts `head -5 ./mappings/disease-mappings.csv`

source,snomedct,orpha,prefname
C0019247,http://purl.bioontology.org/ontology/SNOMEDCT/32895009,https://fake.orpha.net/not-found,Hereditary disease
C0019247,http://purl.bioontology.org/ontology/SNOMEDCT/782964007,https://fake.orpha.net/not-found,Genetic disease
C0235946,http://purl.bioontology.org/ontology/SNOMEDCT/278849000,https://fake.orpha.net/not-found,Cerebral atrophy
C0007760,http://purl.bioontology.org/ontology/SNOMEDCT/223176004,https://fake.orpha.net/not-found,Cerebellar disorder


In [24]:
require 'linkeddata'
require 'csv'

graphing_errors = File.open('./graph/drug-disease-errors.txt', 'w') 

# Define namespaces
SIMPATHIC = RDF::Vocabulary.new('urn:simpathic:')
RDFS = RDF::Vocabulary.new('http://www.w3.org/2000/01/rdf-schema#')

# Read input files
entity_relations = CSV.read('./raw-data/Drug-Diseasetriples.tsv', col_sep: "\t", headers: true)
drug_mappings = CSV.read('./mappings/drug-mappings.csv', headers: true)
disease_mappings = CSV.read('./mappings/disease-mappings.csv', headers: true)

# Create RDF graph
graph = RDF::Graph.new

failures = {}
# Process each entity relation
entity_relations.each do |row|
  # warn row.inspect
  drug_id = row['Drug_id']
  disease_id = row['Disease_id']
  
  # Find corresponding mappings
  drug = drug_mappings.find { |d| d['demokratisid'] == drug_id }
  disease = disease_mappings.find { |d| d['source'] == disease_id }
  
  unless drug
    next if failures[drug_id]
    failures[drug_id] = 1
    warn "drug lookup failed #{drug_id}"
    graphing_errors.write "drug lookup failed #{drug_id}\n"
    next
  end
  unless disease
    next if failures[disease_id]
    failures[disease_id] = 1
    warn "disease lookup failed #{disease_id}"
    graphing_errors.write "disease lookup failed #{disease_id}\n"
    next
  end
  
  # Extract relevant IDs and labels
  # demokratisid,xref,demokratis_label,pubchem_cid,IUPACname
  # C0613621,http://purl.bioontology.org/ontology/MESH/C030536,"2,2-dichloro-1,1-difluoroethyl difluoromethyl ether",https://pubchem.ncbi.nlm.nih.gov/compound/152803,"2,2-dichloro-1,1-difluoroethyl%20difluoromethyl%20ether"
  pubchem_uri = RDF::URI.new(drug['pubchem_cid'])
  pubchem_type = RDF::URI.new("http://semanticscience.org/resource/CHEMINF_000302")
  pubchem_label =  RDF::Literal.new("PubChem Identifier")
  pubchem_core_type = RDF::URI.new("https://w3id.org/biolink/vocab/Drug")
  demokratis_drug_label = RDF::Literal.new(drug['demokratis_label'])
  iupac_drug_label = RDF::Literal.new(drug['IUPACname'])

#   source,snomedct,orpha,prefname
#   C0019247,http://purl.bioontology.org/ontology/SNOMEDCT/32895009,https://fake.orpha.net/not-found,Hereditary disease
  snomed_uri = RDF::URI.new(disease['snomedct'])
  snomed_type = RDF::URI.new("https://bioportal.bioontology.org/ontologies/SNOMEDCT")
  snomed_core_type = RDF::URI.new("https://w3id.org/biolink/vocab/Disease")
  snomed_label =  RDF::Literal.new("SNOMED Term")
  orphanet = RDF::URI.new(disease['orpha'])
  disease_label = RDF::Literal.new(disease['prefname'])
  
  # Create context URI
  context_uri = RDF::URI.new("urn:simpathic:context:#{drug_id}_#{disease_id}")
  
  # Add quads to graph using RDF::Statement
  graph << RDF::Statement.new(pubchem_uri, SIMPATHIC['associated-with'], snomed_uri, graph_name: context_uri)
  
  graph << RDF::Statement.new(pubchem_uri,  RDFS.label,     demokratis_drug_label, graph_name: context_uri)
  graph << RDF::Statement.new(pubchem_uri,  RDFS.label,     iupac_drug_label, graph_name: context_uri)
  graph << RDF::Statement.new(pubchem_uri,  RDF.type,       pubchem_type,          graph_name: context_uri)
  graph << RDF::Statement.new(pubchem_uri,  RDF.type,       pubchem_core_type,             graph_name: context_uri)
  graph << RDF::Statement.new(pubchem_type, RDFS.label,     RDF::Literal.new("PubChem"), graph_name: context_uri)
  graph << RDF::Statement.new(pubchem_core_type, RDFS.label,     RDF::Literal.new("Drug"), graph_name: context_uri)
  graph << RDF::Statement.new(pubchem_uri,  SIMPATHIC['original-id'], RDF::Literal.new("#{drug_id}"), graph_name: context_uri)
 
  graph << RDF::Statement.new(snomed_uri, RDFS.label, disease_label, graph_name: context_uri)
  graph << RDF::Statement.new(snomed_uri, RDF.type, snomed_type, graph_name: context_uri)
  graph << RDF::Statement.new(snomed_uri, RDF.type, snomed_core_type, graph_name: context_uri)
  graph << RDF::Statement.new(snomed_type, RDFS.label, snomed_label, graph_name: context_uri)
  graph << RDF::Statement.new(snomed_uri, SIMPATHIC['orphanet'], orphanet, graph_name: context_uri)
  graph << RDF::Statement.new(snomed_uri, SIMPATHIC['original-id'], RDF::Literal.new("#{disease_id}"), graph_name: context_uri)
  
  graph << RDF::Statement.new(context_uri, SIMPATHIC['skg-source'], RDF::Literal.new("DEMOKRATIS"), graph_name: context_uri)
  graph << RDF::Statement.new(context_uri, SIMPATHIC['skg-evidence'], RDF::Literal.new("TBD"), graph_name: context_uri)
end

# Write RDF to file in N-Quads format
File.open('./graph/drug-disease.nq', 'w') do |f|
  RDF::Writer.for(:nquads).new(f) do |writer|
    writer << graph
  end
end
graphing_errors.close

puts "RDF quads written"

(irb):6: warning: already initialized constant Object::SIMPATHIC
(irb):6: warning: previous definition of SIMPATHIC was here
(irb):7: warning: already initialized constant Object::RDFS
(irb):7: warning: previous definition of RDFS was here
disease lookup failed C0524851
drug lookup failed C0026457
drug lookup failed C1567285
drug lookup failed C0723712
drug lookup failed C3541953
drug lookup failed C0008425
drug lookup failed C0006507
drug lookup failed C0591139
drug lookup failed C0002062
drug lookup failed C0024808
drug lookup failed C0060794
drug lookup failed C0056077
drug lookup failed C3492623
drug lookup failed C0042672
drug lookup failed C2825610
drug lookup failed C0041044
drug lookup failed C0070971
drug lookup failed C0699680
drug lookup failed C0082384
disease lookup failed C0013384
disease lookup failed C0015397
disease lookup failed C0679225
disease lookup failed C0742034
disease lookup failed C0694564
drug lookup failed C0017082
drug lookup failed C0017973
drug lookup fa

RDF quads written


In [18]:
graphing_errors.close